<a href="https://colab.research.google.com/github/Tahahaha7/Quantitative_Trading/blob/main/New_York_Times_Articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip -qq install vaderSentiment
!pip -qq install pynytimes

In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pynytimes import NYTAPI
import datetime as dt
import pandas as pd
from pandas.tseries.offsets import BDay
import warnings
warnings.filterwarnings("ignore")

analyzer = SentimentIntensityAnalyzer()

In [36]:
api_key =  'key_token'

nyt = NYTAPI(api_key, parse_dates=True)

def nyt_query_articles(key_word, n_results, begin, end):
    return nyt.article_search(
        query = key_word,
        results = n_results,
        dates = {"begin": begin, "end": end},
        #options = {https://developer.nytimes.com/docs/articlesearch-product/1/overview}
        )
    
def nyt_query_archives(key_word, n_results, begin, end):
    return nyt.article_search(
        query = key_word,
        results = n_results,
        dates = {"begin": begin, "end": end},
        #options = {https://developer.nytimes.com/docs/articlesearch-product/1/overview}
        )

In [37]:
#articles = nyt_query_archives('Uber', 100, dt.datetime(2015, 1, 1), dt.datetime(2018, 1, 1))
archives = nyt_query_archives('Uber', 100, dt.datetime(2015, 1, 1), dt.datetime(2018, 1, 1))

In [20]:
abstracts = [articles[i]['abstract'] for i in range(len(articles))]
pub_dates = [articles[i]['pub_date'] for i in range(len(articles))]

news = pd.DataFrame({'publishedAt': pub_dates, 'abstract': abstracts})
news.sort_values(by='publishedAt', inplace=True)

def get_trade_open(date):
    curr_date_open = pd.to_datetime(date).floor('d').replace(hour=13, minute=30) - BDay(0)
    curr_date_close = pd.to_datetime(date).floor('d').replace(hour=20, minute=0) - BDay(0)
    
    prev_date_close = (curr_date_open - BDay()).replace(hour=20, minute=0)
    next_date_open = (curr_date_close + BDay()).replace(hour=13, minute=30)
    
    if ((pd.to_datetime(date)>=prev_date_close) & (pd.to_datetime(date)<curr_date_open)):
        return curr_date_open
    elif ((pd.to_datetime(date)>=curr_date_close) & (pd.to_datetime(date)<next_date_open)):
        return next_date_open
    else:
        return None
      
news["trading_time"] = news["publishedAt"].apply(get_trade_open)
final_news = news[pd.notnull(news['trading_time'])]
final_news['Date'] = pd.to_datetime(pd.to_datetime(final_news['trading_time']).dt.date)
final_news.head()

,publishedAt,abstract,trading_time,Date
75,2015-06-01 09:54:08+00:00,The song by an amateur hip-hop artist briefly ...,2015-06-01 13:30:00+00:00,2015-06-01
73,2015-07-10 01:01:03+00:00,JPMorgan Chase became the latest bank to publi...,2015-07-10 13:30:00+00:00,2015-07-10
96,2015-08-04 07:21:04+00:00,Presidential candidates portraying themselves ...,2015-08-04 13:30:00+00:00,2015-08-04
98,2015-08-06 13:00:05+00:00,"By using customers as lobbyists, companies lik...",2015-08-06 13:30:00+00:00,2015-08-06
99,2015-09-02 00:50:24+00:00,The ruling by Judge Edward M. Chen will allow ...,2015-09-02 13:30:00+00:00,2015-09-02
